ICESat-2 orientation can be: backward, forward, and transition. During transition, ICESat-2 is maneuvering between the two orientations. Science quality is potentially degraded while in transition mode. (Source: https://nsidc.org/sites/nsidc.org/files/technical-references/ICESat2_ATL03_data_dict_v005.pdf)


The parameter "/orbit_info/sc_orient" in ATL03 denotes the orientation of ICESat-2.

0 = backward

1 = forward

2 = transition

In [ ]:
import h5py
import pandas as pd

User input: "strong" or "weak"

In [ ]:
def which_beam(f, sc_orient, beam='both'):
    if beam=='both':
        # extracted_beam = 
        print("Subset both beams")
    elif beam=='strong':
        extracted_beam = get_strong_beams(f, sc_orient)
    elif beam=='weak':
        extracted_beam = get_weak_beams(f, sc_orient)
    else:
        print('Input should be one of the following: "both", "strong" or "weak"')
    return extracted_beam

Script reads the variable "/orbit_info/sc_orient" to determine orientation.

In [ ]:
# If input = "strong":
#     If sc_orient = 0:
#         subset gt1l, gt2l, gt3l
#     elif sc_orient = 1:
#         subset gt1r, gt2r, gt3r
#     else:
#         print("Error: transition phase"), or other output?

In [ ]:
def get_strong_beams(f, sc_orient):
    if sc_orient == 0:
        # subset gt1l, gt2l, gt3l
        track = 'gt3l' # or 'gt1l', 'gt2l'
        strong = pd.DataFrame(data={'lat': f[track + '/sea_ice_segments/latitude'][:],
                                  'lon': f[track + '/sea_ice_segments/longitude'][:],
                                  'elev': f[track + '/sea_ice_segments/heights/height_segment_height'][:]
                                 }
                           )
    elif sc_orient == 1:
        # subset gt1r, gt2r, gt3r
        track = 'gt3r' # or 'gt1r', 'gt2r'
        strong = pd.DataFrame(data={'lat': f[track + '/sea_ice_segments/latitude'][:],
                                  'lon': f[track + '/sea_ice_segments/longitude'][:],
                                  'elev': f[track + '/sea_ice_segments/heights/height_segment_height'][:]
                                 }
                           )
    else:
        print("Error: transition phase") # or other output?
        strong = pd.DataFrame()
    return strong

In [ ]:
# If input = "weak":
#     If sc_orient = 0:
#         subset gt1r, gt2r, gt3r
#     elif sc_orient = 1:
#         subset gt1l, gt2l, gt3l
#     else:
#         print("Error: transition phase"), or other output?

In [ ]:
def get_weak_beams(f, sc_orient):
    if sc_orient == 0:
        # subset gt1r, gt2r, gt3r
        track = 'gt3r' # or 'gt1r', 'gt2r'
        weak = pd.DataFrame(data={'lat': f[track + '/sea_ice_segments/latitude'][:],
                                  'lon': f[track + '/sea_ice_segments/longitude'][:],
                                  'elev': f[track + '/sea_ice_segments/heights/height_segment_height'][:]
                                 }
                           )
    elif sc_orient == 1:
        # subset gt1l, gt2l, gt3l
        track = 'gt3l' # or 'gt1l', 'gt2l'
        strong = pd.DataFrame(data={'lat': f[track + '/sea_ice_segments/latitude'][:],
                                  'lon': f[track + '/sea_ice_segments/longitude'][:],
                                  'elev': f[track + '/sea_ice_segments/heights/height_segment_height'][:]
                                 }
                           )
    else:
        print("Error: transition phase") # or other output?
        weak = pd.DataFrame()
    return weak

Sample data to demonstrate idea (Attached in GitHub repo)

In [ ]:
is2_file = 'processed_ATL07-01_20210223050453_09351001_005_01.h5'
with h5py.File(is2_file, 'r') as f:
    is2_gt3r = pd.DataFrame(data={'lat': f['gt3r/sea_ice_segments/latitude'][:],
                                  'lon': f['gt3r/sea_ice_segments/longitude'][:],
                                  'elev': f['gt3r/sea_ice_segments/heights/height_segment_height'][:]
                                 }
                           ) 
    is2_gt3l = pd.DataFrame(data={'lat': f['gt3l/sea_ice_segments/latitude'][:],
                                  'lon': f['gt3l/sea_ice_segments/longitude'][:],
                                  'elev': f['gt3l/sea_ice_segments/heights/height_segment_height'][:]
                                 }
                           ) 
    # orient = pd.DataFrame(data={'lat': f['gt3r/sea_ice_segments/latitude'][:],
    #                             'lon': f['gt3r/sea_ice_segments/longitude'][:],
    #                             'orient': f['orbit_info/sc_orient'][:]
    #                            }
    #                      )

Read in the sample ATL07 data

In [ ]:
is2_file = 'processed_ATL07-01_20210223050453_09351001_005_01.h5'
f = h5py.File(is2_file, 'r')

This sample data I am using only contains gt3l and gt3r

In [ ]:
list(f.keys())

Check the value of sc_orient

In [ ]:
orient = f['orbit_info/sc_orient'][:]
orient

Extract the strong beam only from the sample ATL07 data

In [ ]:
which_beam(f,orient,'strong')

In [ ]:
# When orient==1, ICESat-2 is forward and the Right beams are strong
# Check if gt3r is the output of which_beam(f,orient,'strong')
is2_gt3r